# Data preprocessing

## Check the dataset

Import the packages

In [115]:
import pandas as pd
import numpy as np

In [116]:
df=pd.read_csv('../OriginalDataset/data_weather.csv')

In [117]:
df.head(3)

,date,cloud_cover,sunshine,global_radiation,max_temp,mean_temp,min_temp,precipitation,pressure,snow_depth
0,19790101,2.0,7.0,52.0,2.3,-4.1,-7.5,0.4,101900.0,9.0
1,19790102,6.0,1.7,27.0,1.6,-2.6,-7.5,0.0,102530.0,8.0
2,19790103,5.0,0.0,13.0,1.3,-2.8,-7.2,0.0,102050.0,4.0


In [118]:
df.tail(3)

,date,cloud_cover,sunshine,global_radiation,max_temp,mean_temp,min_temp,precipitation,pressure,snow_depth
15338,20201229,7.0,0.0,21.0,4.1,2.6,1.1,0.0,98830.0,NaN
15339,20201230,6.0,0.4,22.0,5.6,2.7,-0.1,0.0,100200.0,NaN
15340,20201231,7.0,1.3,34.0,1.5,-0.8,-3.1,0.0,100500.0,NaN


In [119]:
print(df.dtypes)

date                  int64
cloud_cover         float64
sunshine            float64
global_radiation    float64
max_temp            float64
mean_temp           float64
min_temp            float64
precipitation       float64
pressure            float64
snow_depth          float64
dtype: object


Find the number of null values for each feature

In [120]:
null_values=df.isnull().sum()
print(null_values)

date                   0
cloud_cover           19
sunshine               0
global_radiation      19
max_temp               6
mean_temp             36
min_temp               2
precipitation          6
pressure               4
snow_depth          1441
dtype: int64


Convert the 'date' to date type

In [121]:
df['date']=pd.to_datetime(df['date'], format='%Y%m%d')
df['date'].head(3)

0   1979-01-01
1   1979-01-02
2   1979-01-03
Name: date, dtype: datetime64[ns]

## Data imputation

### Impute cloud_cover

In [122]:
# Impute NaN in cloud_cover with linear method
df['cloud_cover']=df['cloud_cover'].interpolate(method='linear', limit_direction='both')
df['cloud_cover']=df['cloud_cover'].round(1)

print(df['cloud_cover'].isnull().sum())

0


### Impute global_radiation

In [123]:
# Impute NaN in global_radiation with linear method
df['global_radiation']=df['global_radiation'].interpolate(method='linear', limit_direction='both')
df['global_radiation']=df['global_radiation'].round(1)

print(df['global_radiation'].isnull().sum())


0


### Impute precipitation

In [124]:
# Impute NaN in precipitation with linear method
df['precipitation']=df['precipitation'].interpolate(method='linear', limit_direction='both')
df['precipitation']=df['precipitation'].round(1)

print(df['precipitation'].isnull().sum())

0


### Impute pressure

In [125]:
# Impute NaN in pressure with linear method
df['pressure']=df['pressure'].interpolate(method='linear', limit_direction='both')
df['pressure']=df['pressure'].round(1)

print(df['pressure'].isnull().sum())

0


### Impute temperature

Impute min_temp

In [126]:
# Impute NaN in global_radiation with linear method
df['min_temp']=df['min_temp'].interpolate(method='linear', limit_direction='both')
df['min_temp']=df['min_temp'].round(1)

print(df['min_temp'].isnull().sum())

0


Impute max_temp

In [127]:
# Impute NaN in global_radiation with linear method
df['max_temp']=df['max_temp'].interpolate(method='linear', limit_direction='both')
df['max_temp']=df['max_temp'].round(1)

print(df['max_temp'].isnull().sum())

0


Impute mean_temp

In [128]:
# Find the rows whose value of mean_temp is NaN
null_rows=df[df['mean_temp'].isnull()]

# Calculate the mean value of min_temp and max_temp
for index, row in null_rows.iterrows():
    min_temp=row['min_temp']
    max_temp=row['max_temp']
    mean_temp=np.mean([min_temp, max_temp])
    df.at[index,'mean_temp']=mean_temp

df['mean_temp']=df['mean_temp'].round(1)
print(df['mean_temp'].isnull().sum())

0


### Impute snow_depth

Calculate the number of days whose snow_depth is not zero for each month

In [133]:
df['year']=df['date'].dt.year
df['month']=df['date'].dt.month

snow_days_notzero=df[df['snow_depth'] != 0].groupby(['month'])['snow_depth'].count()
print(snow_days_notzero)

month
1     56
2     48
3      6
4      2
12    28
Name: snow_depth, dtype: int64


We can learn that from May to November, there are no days that have snow_depth and there are only 2 days with snow_depth in April and 6 days in March with forty years. So impute NaN of snow_depth to 0 in these months as follows.

In [132]:
# Impute NaN of snow_depth to 0
months=[3,4,5,6,7,8,9,10,11]
df.loc[(df['month'].isin(months))&(df['snow_depth'].isnull()),'snow_depth']=0

Calculate the days that the 'snow_depth' is NaN of each month.

In [146]:
snow_days_nan=df.groupby(['month'])['snow_depth'].apply(lambda x: x.isnull().sum())
print(snow_days_nan)

month
1     36
2     38
3      0
4      0
5      0
6      0
7      0
8      0
9      0
10     0
11     0
12    38
Name: snow_depth, dtype: int64


In [142]:
df.to_csv('../dataset_for_analysis/data_weather_new.csv',index=0)